<br>

#### week1 ~ the-web-as-filesystem...   &nbsp;&nbsp; (hw1pr3.ipynb)

[the google doc with hw1's details](https://docs.google.com/document/d/11ALzpsANe3ZDR5sk8-kgaElaX_fwlDINVlQ8WS8JfQE/edit)
<hr>

<br>

#### Problem 3:  The "Wisdom of the Web":  Programming the web for inquiry!

(hw1pr3.ipynb)

+ Here, your task is to create a two-hop approach to "inquire about" a subject
  + That is, a request, whose results are used to fashion a new request...
  + whose results are interpreted/parsed into an answer to our original query.

<br>

Below is an example that uses Beautiful Soup -- that is not required.
Feel free to use json-providing APIs (or other APIs providing structured data)

<br>

The assignment page has some example APIs, and there are many (many!) others!
I encourage you to choose 1-2 of interest and explore...
... and, from there, stitch together an "inquiring system"

In [ ]:
#
# see if you have the Beautiful Soup library...
#

import bs4

In [ ]:
#
# If you _don't_ have bs4, try installing it using pip:
#    the pip you want is the one sharing the path of your python interpreter/kernel
#    you can see this by clicking on the upper-left "kernel" pulldown
#        for me, the kernel is /usr/local/bin/python3
#                  so I'll use /usr/local/bin/pip3   (could be pip or pip3)

#    !/usr/local/bin/pip3 install beautifulsoup4

In [ ]:
#
# hopefully, this now works! (if so, it will succeed silently)
#            things to reset: (a) the python + jupyter extensions, (b) vscode itself

import bs4

In [ ]:
#
# this should work from pr1 and pr2...
#

import requests

In [ ]:
#
# this is the most popular HTML parser, lxml -- see if you have it (else install in the same way! :-)

import lxml

<br>
cs35's example :-)
<br>

In [ ]:
#
# get_school_colors_page(school)
#

"""
Here is our strategy:

1. first we grab the pages that define each school's colors (from wikipedia)
2. then, use bs4 to parse those pages and return a list of colors (we only handle pairs of colors)

3. then, grab the page that defines the popularity of colors
4. finally, use bs4 to compute a score for each school based on its colors  (our score == sum)

That's it!  
In each case, you get to decide if it's _higher_ scores or _lower_ scores that are better :-)
"""


from bs4 import BeautifulSoup     

PARSER = "html.parser"     # not as flexible
PARSER = "lxml"            # to use this, install with .../pip install lxml

#
# this gets a school's wikipedia page and then extracts its school colors
#
def get_school_colors_page(school):
    """ get_school_colors_page takes in a string with a formatted  name
        such as Harvey_Mudd_College or Stanford_University
        
        it tried to request the appropriate page from wikipedia and parse
        it with Beautiful Soup - and it should return that soup object
    """
    school_color_url = "http://en.wikipedia.org/wiki/" + school
    response = requests.get(school_color_url)         # request the page
    
    if response.status_code == 404:                 # page not found
        print("For the school", school, "There was a problem with getting the page")
        print(school_color_url)
        
    data_from_url = response.text                   # the HTML text from the page
    soup = BeautifulSoup(data_from_url,PARSER)      # parsed with Beautiful Soup
    return soup


if False:
    #school_2 = 'Harvey_Mudd_College'
    school_2 = 'Pitzer_College'
    # other options that have been tested and work...
    #school_2 = 'Pomona_College'
    #school_2 = 'Scripps_College'
    #school_2 = 'Claremont_McKenna_College'
    #school_2 = 'Stanford_University'
    # not working:
    #school_2 = 'Harvard_University'
    result = get_school_colors_page(school_2) 

In [ ]:
def extract_school_colors(soup):
    """ extract_school_colors takes in a beautiful soup object, soup
        and uses Beautiful Soup to extract a list of all of that school's colors
        
        it return that list of colors
        
        (Note that for different schools, you'll need to run the get_school_colors_page
         to obtain soup objects for each page.)
    """

    AllDivs = soup.findAll('a', attrs={"title":"School colors"})
    # print(len(AllDivs))                 # debugging

    for div in AllDivs:
        ancestor = div.parent.parent      # this is the ancestor that holds the school colors (grandparent)
        TDs = ancestor.findAll('td')      # it's in one of the <td> table cells </td>
        Texts = [ x.text for x in TDs ]   # get the actual text

        if len(Texts) < 1:                # didn't get any? a problem!
            print(f"No colors found!")
            return []

        s = Texts[0]                      # take the first one...
        colors = get_text_with_and(s)     # use our helper function
        return colors                     # the colors... works many times, not every time.


if False:
    colors = extract_school_colors(result)
    colors

In [ ]:
#
# a helper function to handle the messy alternatives, e.g., Black & gold, Blue and white
# 

import string 

DEBUG = True

def get_text_with_and(s):
    """ first, finds 'and' or '&' in s 
        then finds all of the alphabetic 
        characters in either direction from that conjunction!
    """
    allowable = ' '+string.ascii_lowercase  # allowable letters
    s = s.lower()                           # only consider lower case
    if DEBUG: print(s)                                # for debugging
    if ('and' in s or '&' in s) == False:   # is there an 'and' or '&'?  If not...
        print("No 'and' found...")          #    return the lowercased s
        return s

    and_index = s.find('and')               # Try 'and' first
    if and_index == -1:                     # If it's not there,
        and_index = s.find('&')             # Try '&'
        if and_index == -1:                 # This should have been caught, but I'm superstitious
            print("No 'and' found...")
            return s
        left = s[:and_index]                # get the left and right pieces
        right = s[and_index+1:]             # '&' is one char long, hence the 1 
    else:
        left = s[:and_index]                # get the left and right pieces
        right = s[and_index+3:]             # 'and' is three chars long, hence the 3

    # print(f"left, right are {left,right}") # for debugging -- now, get the color on the left and right:

    new_left = ''
    for c in left[::-1]:   # go backwards
        if c not in allowable:
            break
        else:
            new_left = c + new_left          # accumulate, as long as it's a character
    left = new_left

    new_right = ''
    for c in right[:]:   # go forwards
        if c not in allowable:
            break
        else:
            new_right = new_right + c        # accumulate, as long as it's a character
    right = new_right

    left = left.strip()                      # remove whitespace on either side
    right = right.strip()                    # remove whitespace on either side
    #print(f"left, right are now {left,right}")  # more debugging!

    return [left, right]                     # only two colors for this scraper!

if False:
    result = get_text_with_and("#$ this & that !?")
    print(f"result is {result}")

In [ ]:
#
# Color-page handling
# 

# this line imports the HTML parser named BeautifulSoup:
#      it's a class constructor; when called it will return an object of type BeautifulSoup
from bs4 import BeautifulSoup     

PARSER = "html.parser"     # not as flexible a parser
PARSER = "lxml"            # to use lxml (the most common), you'll need to install with .../pip install lxml

#
# get_color_page()
#
def get_color_page():
    """ This function requests the most-popular-colors page
        and parses it with Beautiful Soup, returning the resulting
        Beautiful Soup object, soup
    """
    color_popularity_url = "http://www.thetoptens.com/top-ten-favorite-colors/"
    response = requests.get(color_popularity_url)   # request the page

    if response.status_code == 404:                 # page not found
        print("There was a problem with getting the page:")
        print(color_popularity_url)

    data_from_url = response.text                   # the HTML text from the page
    #soup = BeautifulSoup(data_from_url,"lxml")      # parsed with Beautiful Soup
    soup = BeautifulSoup(data_from_url,PARSER)      # parsed with Beautiful Soup
    return soup


#
# find_color_score( color, soup )
#
def find_color_score( color_name, soup ):
    """ find_color_score takes in color_name (a string represnting a color)
        and soup, a Beautiful Soup object returned from a successful run of
        get_color_page
        
        find_color_score returns our predictive model's number of points in
        a potential match up involving a team with that color
        
        the number of points is 21 - ranking, where ranking is from 1 (most
        popular color) to 20 (least popular color) or 21, representing all
        of the others
    """
    ListOfDivs = soup.findAll('div', {'class':"i"})   # the class name happens to be 'i' here...
    ranking = 1
    for div in ListOfDivs:
        # print(div.em, div.b)                    # checking the subtags named em and b
        this_divs_color = div.b.text.lower()      # getting the text from them (lowercase)
        this_divs_ranking = 21                    # the deafult (integer) ranking: 21
        try:
            this_divs_ranking = int(ranking)      # try to convert it to an integer
        except:                                   # if it fails
            pass                                  # do nothing and leave it at 21
        
        if color_name == this_divs_color:         # check if we need to return this one
            return this_divs_ranking
        ranking +=1
    # if we ran through the whole for loop without finding a match, the ranking is 21
    return 21


if True:
    color_soup = get_color_page()      # should run this once!
    color_to_test = "blue"
    score = find_color_score(color_to_test,color_soup)
    print(f"score for {color_to_test} is {score}")




In [ ]:

#
# put it all together!
#
if True:
    """
    # Here is an example of using the Web's Wisdom to answer the question
    #
    #     Which college is best.....?
    #
    #     Not limited to the 5Cs!
    """
    #school_1 = 'Harvey_Mudd_College'
    #school_1 = 'Pitzer_College'
    school_1 = 'Scripps_College'
    #other options (that have been tested and work)...
    school_2 = 'Pomona_College'
    #school_2 = 'Claremont_McKenna_College'
    #school_2 = 'Stanford_University'
    #school_2 = 'Harvard_University'
    # 
    # As a reminder, here is our "web scavenging" approach:
    #
    # 1. first we grab the pages that define each school's colors (wikipedia)
    # 2. then, use bs4 to parse those pages and return a list of colors
    # 3. then, grab the page that defines the popularity of colors
    # 4. finally, use bs4 to compute a score for each school based on its colors
    #

    # we get the school colors page for each school
    # and we return a BeautifulSoup "soup" object for each!
    school_soup_1 = get_school_colors_page(school_1)
    school_soup_2 = get_school_colors_page(school_2)
    print("Done scraping the school colors.\n")

    # We have a function that actually grabs the colors from the page...
    school_colors_1 = extract_school_colors( school_soup_1 )
    school_colors_2 = extract_school_colors( school_soup_2 )
    print()
    print("School 1 (" + school_1 + ") colors:", school_colors_1)
    print("School 2 (" + school_2 + ") colors:", school_colors_2)


    # Next, we grab the color-popularity page (and parse it into
    # a BeautifulSoup object...
    # 
    color_popularity_soup = get_color_page()
    print("\nDone scraping the color-popularity page.\n")

    # Finally, we convert the team colors into total scores
    # which will reveal our predicted result
    # Admittedly, our "points" are simply the ranking of how popular a color is.

    # let's use a list comprehension as a reminder of how those work...
    school_1_scores = [ find_color_score(clr, color_popularity_soup) for clr in school_colors_1 ]
    school_2_scores = [ find_color_score(clr, color_popularity_soup) for clr in school_colors_2 ]
    print("School 1 (" + school_1 + ") scores:", school_1_scores)
    print("School 2 (" + school_2 + ") scores:", school_2_scores)
    print()
    print("School 1 (" + school_1 + ") total score:", sum(school_1_scores))
    print("School 2 (" + school_2 + ") total score:", sum(school_2_scores))

    #find the winner
    if sum(school_1_scores) > sum(school_2_scores):
        print(school_1, " wins!")
    elif sum(school_1_scores) < sum(school_2_scores):
        print(school_2, " wins!")
    else:
        print("It's a tie!")


In [ ]:
#
# your scavenger hunt - or question-answering - need not 
#

In [ ]:
#
# Your task!     
#

#
# Beautiful soup is not required!  Feel free to try it, if you'd like...
# 
# Your task is to choose 1-2 new web APIs (or web pages) and then
# create a two-step process that answers a question 
#      (as you see, both serious and not-so-serious questions are welcome!)
# 
# That is, there should be a web-access (API or beautiful-soup-scraping) to obtain a first piece of information
# then, use that to create a second web-access (same API, different API, or another raw page)
# from which you construct your answer!
#
# As you see in this example, it does not _always_ need to work (Aargh! Harvard! ;-)
#
# But, it should work for several examples -- show off with a few of them :-)
# 



<br>

#### Your wisdom-of-the-web answerer ...


<br>

In [ ]:
#
#
#